In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height: 10000px;
}
</style>

In [ ]:
from notebooks.exercises.src.text.news_wiki_dashboard import RSSWikiDashboard

In [ ]:
from src.text.document_retrieval.wikipedia import Wikipedia

from notebooks.exercises.src.text.news_wiki_search_init import RsspediaInit

In [ ]:
wikipedia = Wikipedia(
    language="Danish",
    cache_directory_url=False
)
rss_search_init = RsspediaInit(wikipedia = wikipedia)

In [ ]:
rsswdb = RSSWikiDashboard(wikipedia, rss_search_init)
rsswdb.start

In [ ]:
r = rsswdb.rsspedia_search.cdist_func([rss_search_init.sumVectorRepresentation("6")],
    [rss_search_init.sumVectorRepresentation("Guide Anmelderne anbefaler 6 gode spisesteder med pasta på menuen")])

In [ ]:
import numpy as np

text = "10"
n_matches = 3
p = 0.5

ngrams = rsswdb.rsspedia_search.get_ngrams(text)

r = []
titles = [] 
texts = []
urls = []
scores = []
        
for i in range(len(ngrams)):
    cdist_result = rsswdb.rsspedia_search.cdist_func(rsswdb.rsspedia_search.wikipedia_title_vectors, [rsswdb.rsspedia_search.rsspediainit.sumVectorRepresentation(text = ngrams[i], type = rsswdb.rsspedia_search.embedding_composition)])
    cdist_result2 = rsswdb.rsspedia_search.cdist_func([rsswdb.rsspedia_search.rsspediainit.sumVectorRepresentation(text = text, type = rsswdb.rsspedia_search.embedding_composition)], [rsswdb.rsspedia_search.rsspediainit.sumVectorRepresentation(text = ngrams[i], type = rsswdb.rsspedia_search.embedding_composition)])

    cdist_list1 = cdist_result[1] # List of all the cosine distances
    cdist_list2 = cdist_result2[1]
    cdist_list = (cdist_list1 * p + cdist_list2 * (1 - p))
    cdist_list_sorted = np.sort(cdist_list, axis = 0) # Sorted list of cosine distances - to get top N matches

    for j in range(n_matches):
        if(cdist_list[x]):
            x = np.where(cdist_list == cdist_list_sorted[j])[0]
        else:
            continue

In [ ]:
# Transform into list of tuples
r = [ (r[i][0][0], r[i][1][0]) for i in range(len(r))]
# Sort the list of tuples by cosine distance
r = sorted(r, key=lambda tup: tup[1])

for i in range(len(r)):
    document = rsswdb.rsspedia_search.wikipedia.documents_clean[r[i][0]]
    titles.append(document.title)
    #print("{} {}".format(document.title, r[i][1]))
    texts.append(document.abstract)
    urls.append(document.url)
    scores.append(r[i][1])

In [ ]:
import pprint
pprint.pprint(titles)